### Plot NLLH vs number of training samples

This notebook creates plots which show the achieved NLLH vs the number of runtime observations per problem instance used for training the model. It expects to find pickled predictions for 10 folds and different seeds with a different number of training samples with the following filenames:

 * ```*_<number_of_training_samples>/<scenario_name>.floc.<dist_name>.<fold>.<seed>.<model>.pkl```

`eval_model.py` automatically creates such files. 

To keep computation time low, this notebook uses joblib to parallelize computation across all available cores

**NOTE:** The first cell contains some variables that need to be adjusted. Evaluating all scenarios might take a while.

In [ ]:
import glob
import os
import pickle
import sys

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import scipy.stats as scst
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import tabulate
import time

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

% matplotlib inline

sys.path.append("../")
from helper import load_data, preprocess, data_source_release

# 1) Set these paths
# define output_dir such that there are <output_dir>_<numsamples> directories
output_dir = "../TEST" 
save_dir = "../results/"

distribution_ls = ['lognormal', 'invgauss', 'expon']
distribution_hndl_ls = [scst.distributions.lognorm, scst.distributions.invgauss, scst.distributions.expon, ]

# 2) Manually list available subset sizes (must match dirnames used above)
steps = [50, 100]

# 3) Select a scenario
SCENARIO = 'clasp_factoring'
#SCENARIO = 'lpg-zeno'
#SCENARIO = 'yalsat_smallworlds'
#SCENARIO = 'spear_smallworlds'
#SCENARIO = 'saps-CVVAR'
#SCENARIO = 'spear_qcp'
#SCENARIO = 'yalsat_qcp'

#config = "DEFAULT"
LOAD = False

In [ ]:
res_dict = dict()

if LOAD and os.path.isfile(save_dir + "/subsets_%s_result_dict.pkl" % SCENARIO):
    with open(save_dir + "/subsets_%s_result_dict.pkl" % SCENARIO, 'rb') as fh:
        res_dict = pickle.load(fh)
    print("LOADED")
else:
    dirs = glob.glob(output_dir + '_*')
    print(dirs)
    for step_dir in dirs:
        if "_results"  in step_dir:
            continue
        step = int(float(step_dir.split('_')[-1]))
        print(step)
        for fl in os.listdir(step_dir): # + "/%s/" % config):
            if SCENARIO not in fl:
                continue

            fl_path = os.path.join(step_dir, fl)

            if os.path.isdir(fl_path):
                continue

            if "txt" in fl_path or "png" in fl_path:
                continue
            
            if "_result_dict.pkl" in fl or "_nllh_dict.pkl" in fl:
                continue
                
            if not "pkl" in fl:
                continue

            with open(fl_path, "rb") as fh:
                pkl = pickle.load(fh)
                train_pred = np.array(pkl[0])
                val_pred = np.array(pkl[1])
                add_info = pkl[2]
                task = add_info["task"]
                model = add_info["model"]
                scenario = add_info["scenario"]
                assert scenario == SCENARIO
                fold = add_info["fold"]
                seed = add_info["seed"]
                num_samples = add_info["num_train_samples"]
                assert step == num_samples, (fl_path, step, num_samples)

            if task == 'mean':
                continue

            if model not in res_dict:
                res_dict[model] = dict()
                for d, hdl in zip(distribution_ls, distribution_hndl_ls):
                    if d in model:
                        res_dict[model]['dist'] = (d, hdl)
            assert 'dist' in res_dict[model], (res_dict[model], model, dist, fl_path)

            if step not in res_dict[model]:
                res_dict[model][step] = dict()

            if fold not in res_dict[model][step]:
                res_dict[model][step][fold] = {
                    "train": list(),
                    "val": list()
                }

            res_dict[model][step][fold]["train"].append(train_pred)
            res_dict[model][step][fold]["val"].append(val_pred)

    with open(save_dir + "/subsets_%s_result_dict.pkl" % SCENARIO, 'wb') as fh:
        pickle.dump(res_dict, fh)
        print("dumped to " + save_dir + "/subsets_%s_result_dict.pkl" % SCENARIO)

In [ ]:
def comput_nllh_per_inst(obs, p, dist_hdl):
    if dist_hdl.name == "expon" and type(p) == np.float64: p = [p, ]
    
    if len(p) == 1: p = [0, p[0]]
    elif len(p) == 2: p = [p[0], 0, p[1]]
    else: pass
    #print(dist_hdl, p)
    assert p[-2] == 0
    nllh_per_inst = dist_hdl.logpdf(obs, *p[:-2], loc=p[-2], scale=p[-1]) 
    nllh_per_inst += np.log(max(obs))              
    nllh_per_inst = np.mean(-nllh_per_inst)
    return nllh_per_inst

def compute_nllh(obss, ps, dist_hdl):
    nllh = list()
    for obs, p in zip(obss, ps):
        nllh.append(comput_nllh_per_inst(obs, p, dist_hdl))
    return np.mean(nllh)

def compute_nllh_for_all_steps(y_run, p_list, dist_hdl, n_seeds=10, steps=[1, 2, 4, 8, 16, 32, 64, 100]):
    r_dict = dict()
    
    for s_idx, step in enumerate(steps):
        nllh = [compute_nllh(y_run, p_list[s_idx][seed], dist_hdl) for seed in range(n_seeds)]
        nllh = np.array(nllh).reshape([n_seeds, ])
        r_dict[step] = nllh
    return r_dict

In [ ]:
if LOAD and os.path.isfile(save_dir + "/subsets_%s_nllh_dict.pkl" % SCENARIO):
    with open(save_dir + "/subsets_%s_nllh_dict.pkl" % SCENARIO, 'rb') as fh:
        nllh_dict = pickle.load(fh)
    print("LOADED")
else:
    n_seeds = 10

    # 1) Load data
    sc_dict = data_source_release.get_sc_dict()
    data_dir = data_source_release.get_data_dir()

    runtimes, features, sat_ls = load_data.\
        get_data(scenario=SCENARIO, data_dir=data_dir,
                 sc_dict=sc_dict, retrieve=sc_dict[SCENARIO]['use'])
    features = np.array(features)
    runtimes = np.array(runtimes)

    # We do not need any features
    del features

    model_list = sorted(res_dict.keys())
    nllh_dict = dict()
    for model in model_list:
        nllh_dict[model] = dict()

        for step in steps:
            nllh_dict[model][step] = dict()

            for fold in range(10):
                nllh_dict[model][step][fold] = np.zeros([n_seeds, 2]) + 100

    # Get CV splits
    idx = list(range(runtimes.shape[0]))
    kf = KFold(n_splits=10, shuffle=True, random_state=0)
    fold = -1
    for train, valid in kf.split(idx):
        fold += 1

        y_tra_run = runtimes[train]
        y_val_run = runtimes[valid]  

        y_max_ = np.max(y_tra_run.flatten())
        y_min_ = 0
        y_tra_run = (y_tra_run - y_min_) / y_max_
        y_val_run = (y_val_run - y_min_) / y_max_

        start = time.time()
        with joblib.Parallel(n_jobs=-1) as para:
            tra_nllh = para(joblib.delayed(compute_nllh_for_all_steps)(
                y_tra_run, 
                [res_dict[model][step][fold]["train"] for step in steps], 
                res_dict[model]['dist'][1], n_seeds=n_seeds, steps=steps) for model in model_list)

            val_nllh = para(joblib.delayed(compute_nllh_for_all_steps)(
                y_val_run, 
                [res_dict[model][step][fold]["val"] for step in steps], 
                res_dict[model]['dist'][1], n_seeds=n_seeds, steps=steps) for model in model_list)
        end = time.time()
        print("Fold %d took %g sec" % (fold, end - start))
        # now insert results
        for m_idx, model in enumerate(model_list):
            for step in steps:
                nllh_dict[model][step][fold][:, 0] = tra_nllh[m_idx][step]
                nllh_dict[model][step][fold][:, 1] = val_nllh[m_idx][step]

    with open(save_dir + "/subsets_%s_nllh_dict.pkl" % SCENARIO, 'wb') as fh:
        pickle.dump(nllh_dict, fh)
        print("dumped to " + save_dir + "/subsets_%s_nllh_dict.pkl" % SCENARIO)

In [ ]:
models = sorted(list(nllh_dict))
models = [[
           "expon_distfit", 
           "expon_distfit.rf", "expon_distfit.rf2",
           "expon_nn"
          ],
          [
           "invgauss_distfit", 
           "invgauss_distfit.rf", "invgauss_distfit.rf2",
           "invgauss_nn"
          ],
          [
           "lognormal_distfit",
           "lognormal_distfit.rf", "lognormal_distfit.rf2",
           "lognormal_nn"
          ]]

style_dict = {"expon_distfit":         ("EXP fitted",  "", '#d95f02', 'o', 0), 
              "expon_distfit.rf":      ("EXP RF",      '#33a02c', "", 'x', 1),
              "expon_distfit.rf2":     ("EXP iRF",     '#1f78b4', "", '+', 2),
              "expon_nn":              ("EXP DistNet", "", '#252525', 'v', 3),
              "invgauss_distfit":      ("INV fitted",  "", '#d95f02', 'o', 0),
              "invgauss_distfit.rf":   ("INV RF",      '#33a02c', "", 'x', 1),
              "invgauss_distfit.rf2":  ("INV iRF",     '#1f78b4', "", '+', 2),
              "invgauss_nn":           ("INV DistNet", "", '#252525', 'v', 3),
              "lognormal_distfit":     ("LOG fitted",  "", '#d95f02', 'o', 0),
              "lognormal_distfit.rf":  ("LOG RF",      '#33a02c', "", 'x', 1),
              "lognormal_distfit.rf2": ("LOG iRF",     '#1f78b4', "", '+', 2),
              "lognormal_nn":          ("LOG DistNet", "", '#252525', 'v', 3),
}

scenario_dict = {
    "clasp_factoring": r"\textit{Clasp}-\textit{factoring}",
    "lpg-zeno": r"\textit{LPG}-\textit{Zenotravel}",
    "probsat-7sat90": r"\textit{ProbSAT}-\textit{7SAT}",
    "yalsat_smallworlds": r"\textit{YalSAT}-\textit{Small}",
    "yalsat_qcp-hard": r"\textit{YalSAT}-\textit{QCP-hard}",
    "yalsat_qcp-backup": r"\textit{YalSAT}-\textit{QCP}",
    "yalsat_bqcp": r"\textit{YalSAT}-\textit{BQCP}",
    "spear_smallworlds": r"\textit{Spear}-\textit{Small}",
    "spear_qcp-hard": r"\textit{Spear}-\textit{QCP-hard}",
    "spear_qcp-backup": r"\textit{Spear}-\textit{QCP}",
    "spear_bqcp": r"\textit{Spear}-\textit{BQCP}",
    "clasp-K5": r"\textit{Clasp}-\textit{K5}",
    "saps-CVVAR": r"\textit{saps}-\textit{CVVAR}"
}

range_dict = {
    "clasp_factoring": [-0.5, 0.5],
    "lpg-zeno": [-1, 1],
    "probsat-7sat90": [-0.6, 1],
    "yalsat_qcp-hard": [-0.81, 0],
    "spear_smallworlds": [-1, 3],
    "spear_qcp-hard": [-1.2, 0],
    "clasp-K5": [-1.5, 1],
    "saps-CVVAR": [-1, 100]
}

for series in models:
    fig = plt.figure(figsize=[10, 5])
    a = fig.add_subplot(121)
    b = fig.add_subplot(122, sharey=a)
    for model in series:
        for step in nllh_dict[model]:
            for fold in range(10):
                if step == 1 and fold == 0:
                    a.scatter([step, ] * 10,
                              nllh_dict[model][step][fold][:, 0], 
                              label=style_dict[model][0],
                              facecolor=style_dict[model][1], 
                              edgecolor=style_dict[model][2], 
                              marker=style_dict[model][3])
                else:
                    a.scatter([step, ] * 10, 
                              nllh_dict[model][step][fold][:, 0], 
                              facecolor=style_dict[model][1], 
                              edgecolor=style_dict[model][2], 
                              marker=style_dict[model][3])
                b.scatter([step, ] * 10,
                          nllh_dict[model][step][fold][:, 1],
                          facecolor=style_dict[model][1], 
                          edgecolor=style_dict[model][2], 
                          marker=style_dict[model][3])

    a.set_title('{:s} / Train instances'.format(scenario_dict[SCENARIO]))
    a.legend()
    a.set_xlabel('# samples per instance')
    a.set_ylabel('nllh')
    yl = a.get_ylim()
    a.set_ylim(range_dict.get(SCENARIO, [max(-1, yl[0]), min(2, yl[1])]))
    a.set_xlim([0.9, 150])
    a.set_xscale('log')
    
    b.set_title('Test instances')
    b.set_xlabel('# samples per instance')
    b.set_ylabel('nllh')
    b.set_xlim([0.9, 150])
    b.set_xscale('log')
    
    fig.savefig(save_dir + '%s_%s_subset.png' % (SCENARIO, model.split('_')[0]))
    plt.show()

In [ ]:
legend_fs = 16
label_fs = 18
tick_fs = 20
title_fs = 16

for series in models:
    fig = plt.figure(figsize=[5, 5])
    a = fig.add_subplot(111)
    for model in series:
        test_mean = list()
        test_lower = list()
        test_upper = list()
        steps = list()
        

        for step in sorted(nllh_dict[model]):
            tst_m = list()
            tst_l = list()
            tst_u = list()
            for fold in range(10):
                tst_m.append(np.mean(nllh_dict[model][step][fold][:, 1]))
            
            tst_u = np.mean(tst_m) + np.std(tst_m)
            tst_l = np.mean(tst_m) - np.std(tst_m)
            tst_m = np.mean(tst_m)

            test_mean.append(tst_m)
            test_lower.append(tst_l)
            test_upper.append(tst_u)
            steps.append(step)
    
        if model in ("expon_distfit", "lognormal_distfit", "invgauss_distfit"):
            print(test_mean[-1])
            steps = [steps[0], steps[-1]]
            test_mean = [test_mean[-1], test_mean[-1]]
            test_lower = [test_lower[-1], test_lower[-1]]
            test_upper = [test_upper[-1], test_upper[-1]]
    
            a.plot(steps,
                       test_mean, 
                       c = style_dict[model][2] if style_dict[model][1]=='' else style_dict[model][1],
                       marker='',
                       label=style_dict[model][0],
                      )
            a.fill_between(steps, test_lower, test_upper, 
                           color=style_dict[model][2] if style_dict[model][1]=='' else style_dict[model][1],
                           alpha=0.2)
        else:
            a.plot(steps,
                   test_mean, 
                   c = style_dict[model][2] if style_dict[model][1]=='' else style_dict[model][1],
                   marker=style_dict[model][3],
                   label=style_dict[model][0],
                  )
            a.fill_between(steps, test_lower, test_upper, 
                           color=style_dict[model][2] if style_dict[model][1]=='' else style_dict[model][1],
                           alpha=0.2)
            

    a.set_title('{:s}'.format(scenario_dict[SCENARIO]), fontsize=title_fs)
    a.legend(fontsize=legend_fs)
    a.set_xlabel(r'\#samples per train instance', fontsize=label_fs)
    #a.set_ylabel(r'$\frac{1}{|\Pi|} \sum_{\pi \in \Pi} - \log \left( \mathcal{L}_{\mathcal{D}}(\theta \mid t(\pi)_1, ..., t(\pi)_k) \cdot \max_{i\in \{1 ... k\}} t(\pi)_i \right)$')
    a.set_ylabel("nllh (test)", fontsize=label_fs)
    yl = a.get_ylim()
    a.set_ylim(range_dict.get(SCENARIO, [max(-1, yl[0]), min(2, yl[1])]))
    a.set_xlim([0.9, 150])
    a.set_xscale('log')
    a.tick_params(axis='both', which='major', labelsize=tick_fs)
    plt.tight_layout()
    fig.savefig(save_dir + '/pretty_%s_%s_subset.png' % (SCENARIO, model.split('_')[0]))
    plt.show()

In [ ]:
legend_fs = 16
label_fs = 25
tick_fs = 25
title_fs = 16
markersize = 10

new_models = [
          #[
          # "expon_distfit", 
          # "expon_distfit.rf", "expon_distfit.rf2",
          # "expon_nn"
          #],
          #[
          # "invgauss_distfit", 
          # "invgauss_distfit.rf", "invgauss_distfit.rf2",
          # "invgauss_nn"
          #],
          [
           "lognormal_distfit", #"invgauss_distfit",
           "lognormal_distfit.rf", #"invgauss_distfit.rf",
           "lognormal_nn", #"invgauss_nn"
          ]]

new_style_dict = {"invgauss_distfit":      ("INV fitted",  "", '#d95f02', 'o', '-', 0),
                  "invgauss_distfit.rf":   ("INV mRF",      '#33a02c', "", 'x', '-', 1),
                  #"invgauss_distfit.rf2":  ("INV iRF",     '#1f78b4', "", '+', 2),
                  "invgauss_nn":           ("INV DistNet", "", '#252525', 'v', '-', 3),
                  "lognormal_distfit":     ("LOG fitted",  "", '#d95f02', 'o', ':', 0),
                  "lognormal_distfit.rf":  ("LOG mRF",      '#33a02c', "", 'x', ':', 1),
                  #"lognormal_distfit.rf2": ("LOG iRF",     '#1f78b4', "", '+', 2),
                  "lognormal_nn":          ("LOG DistNet", "", '#252525', 'v', ':', 3),
}

new_range_dict = {
    "clasp_factoring": [-0.5, 0.5],
    "lpg-zeno": [-1, 1],
    "probsat-7sat90": [-0.6, 1],
    "yalsat_qcp-hard": [-0.81, -0.1],
    "spear_smallworlds": [-1, 3],
    "spear_qcp-hard": [-1.2, 0],
    "clasp-K5": [-1.5, 1],
    "saps-CVVAR": [-1, 100]
}

for series in new_models:
    fig = plt.figure(figsize=[5, 5])

    a = fig.add_subplot(111)
    for model in series:
        test_mean = list()
        test_lower = list()
        test_upper = list()
        steps = list()

        for step in sorted(nllh_dict[model]):
            if model == "lognormal_distfit.rf" and step == 1:
                continue
            tst_m = list()
            tst_l = list()
            tst_u = list()
            for fold in range(10):
                tst_m.append(np.mean(nllh_dict[model][step][fold][:, 1]))
            #tst_l = np.percentile(tst_m, 5)
            #tst_u = np.percentile(tst_m, 95)
            #tst_m = np.median(tst_m)
            
            tst_u = np.mean(tst_m) + np.std(tst_m)
            tst_l = np.mean(tst_m) - np.std(tst_m)
            tst_m = np.mean(tst_m)

            test_mean.append(tst_m)
            test_lower.append(tst_l)
            test_upper.append(tst_u)
            steps.append(step)
    
        if model in ("expon_distfit", "lognormal_distfit", "invgauss_distfit"):
            print(test_mean[-1])
            steps = [steps[0], steps[-1]]
            test_mean = [test_mean[-1], test_mean[-1]]
            test_lower = [test_lower[-1], test_lower[-1]]
            test_upper = [test_upper[-1], test_upper[-1]]
    
            l = a.plot(steps,
                       test_mean, 
                       c = new_style_dict[model][2] if new_style_dict[model][1]=='' else new_style_dict[model][1],
                       marker='',
                       label=new_style_dict[model][0],
                       linestyle=new_style_dict[model][4], linewidth=3, markersize=markersize
                      )
            a.fill_between(steps, test_lower, test_upper, 
                           color=new_style_dict[model][2] if new_style_dict[model][1]=='' else new_style_dict[model][1],
                           alpha=0.2)
        else:
            l = a.plot(steps,
                   test_mean, 
                   c = new_style_dict[model][2] if new_style_dict[model][1]=='' else new_style_dict[model][1],
                   marker=new_style_dict[model][3],
                   label=new_style_dict[model][0],
                   linestyle=new_style_dict[model][4], linewidth=3, markersize=markersize
                  )
            a.fill_between(steps, test_lower, test_upper, 
                           color=new_style_dict[model][2] if new_style_dict[model][1]=='' else new_style_dict[model][1],
                           alpha=0.2)
            

    #a.set_title('{:s}'.format(scenario_dict[SCENARIO]), fontsize=title_fs)
    
    #a.legend(fontsize=legend_fs)
    
    
    a.set_xlabel(r'\#samples per train instance', fontsize=label_fs)
    #a.set_ylabel(r'$\frac{1}{|\Pi|} \sum_{\pi \in \Pi} - \log \left( \mathcal{L}_{\mathcal{D}}(\theta \mid t(\pi)_1, ..., t(\pi)_k) \cdot \max_{i\in \{1 ... k\}} t(\pi)_i \right)$')
    #a.set_ylabel("nllh (test)", fontsize=label_fs)
    yl = a.get_ylim()
    a.set_ylim(new_range_dict.get(SCENARIO, [max(-1, yl[0]), min(2, yl[1])]))
    a.set_xlim([0.9, 110])
    a.set_xscale('log')
    a.tick_params(axis='both', which='major', labelsize=tick_fs)
    plt.tight_layout()
    fig.savefig(save_dir + '/pretty_%s_%s_subset_special.png' % (SCENARIO, model.split('_')[0]))
    plt.show()